<a href="https://colab.research.google.com/github/Vandana-s-22/intelproject/blob/main/intelproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 51.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [ ]:
pip install PyMuPDF

In [ ]:
pip install fpdf

In [ ]:
%%writefile vandana.py
import streamlit as st
import fitz  # PyMuPDF
from fpdf import FPDF
import re
from faker import Faker
import random
import tempfile
from difflib import SequenceMatcher

# Initialize Faker
fake = Faker()

# Template for comparison
template_contract = f"""
BUSINESS CONTRACT

PARTIES INVOLVED:
  This contract is between {{company_a}} (hereinafter referred to as 'Party A') and {{company_b}} (hereinafter referred to as 'Party B').

CONTRACT DURATION:
  This contract shall commence on {{start_date}} and shall continue until {{end_date}} unless terminated earlier in accordance with the terms herein.

PAYMENT TERMS:
  Payment Terms.1: {{sentence}}
  Payment Terms.2: {{sentence}}
  Payment Terms.3: {{sentence}}

CONFIDENTIALITY:
  Confidentiality.1: {{sentence}}
  Confidentiality.2: {{sentence}}
  Confidentiality.3: {{sentence}}

TERMINATION:
  Termination.1: {{sentence}}
  Termination.2: {{sentence}}
  Termination.3: {{sentence}}

LIABILITY:
  Liability.1: {{sentence}}
  Liability.2: {{sentence}}
  Liability.3: {{sentence}}

DISPUTE RESOLUTION:
  Dispute Resolution.1: {{sentence}}
  Dispute Resolution.2: {{sentence}}
  Dispute Resolution.3: {{sentence}}

MISCELLANEOUS:
  Miscellaneous.1: {{sentence}}
  Miscellaneous.2: {{sentence}}
  Miscellaneous.3: {{sentence}}

SIGNATURES:
  Party A Representative: {{name}}
  Party B Representative: {{name}}

Date: {{date}}
"""

# Generate random contract text for template
def generate_template_contract():
    return template_contract.format(
        company_a=fake.company(),
        company_b=fake.company(),
        start_date=fake.date_this_year(),
        end_date=fake.date_this_decade(),
        sentence=fake.sentence(),
        name=fake.name(),
        date=fake.date_this_year()
    )

# Function to save contract as PDF with style
def save_contract_as_pdf(contract_text, file_name):
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Business Contract', 0, 1, 'C')

        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.set_fill_color(200, 220, 255)
            self.cell(0, 10, title, 0, 1, 'L', 1)
            self.ln(4)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()

        def add_contract(self, contract_text):
            for line in contract_text.split('\n'):
                if line.strip() == '':
                    continue
                if line.isupper():
                    self.chapter_title(line)
                else:
                    self.chapter_body(line)

    pdf = PDF()
    pdf.add_page()
    pdf.set_left_margin(10)
    pdf.set_right_margin(10)
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font('Arial', '', 12)

    pdf.add_contract(contract_text)
    pdf.output(file_name)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Function to summarize contract by extracting key sections
def summarize_contract(text):
    summary = {}

    sections = {
        'parties': r'PARTIES INVOLVED:\s*(.*?)\n',
        'duration': r'CONTRACT DURATION:\s*(.*?)\n',
        'payment_terms': r'PAYMENT TERMS:\s*(.*?)\nCONFIDENTIALITY',
        'confidentiality': r'CONFIDENTIALITY:\s*(.*?)\nTERMINATION',
        'termination': r'TERMINATION:\s*(.*?)\nLIABILITY',
        'liability': r'LIABILITY:\s*(.*?)\nDISPUTE RESOLUTION',
        'dispute_resolution': r'DISPUTE RESOLUTION:\s*(.*?)\nMISCELLANEOUS',
        'miscellaneous': r'MISCELLANEOUS:\s*(.*?)\nSIGNATURES',
        'signatures': r'SIGNATURES:\s*(.*?)\n'
    }

    for key, pattern in sections.items():
        match = re.search(pattern, text, re.DOTALL)
        summary[key] = match.group(1).strip() if match else 'Not found'

    return summary

# Function to check deviation from the template
def check_deviation(text, template):
    sm = SequenceMatcher(None, text, template)
    deviation_ratio = sm.ratio()
    deviation_percentage = (1 - deviation_ratio) * .001
    return deviation_percentage

# Function to highlight important details in the contract PDF
def highlight_important_details(pdf_path, highlights, output_path):
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        for highlight in highlights:
            text_instances = page.search_for(highlight)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.update()
    document.save(output_path)

# Streamlit app
st.title("Business Contract Generator and Validator")

st.sidebar.title("Options")
option = st.sidebar.selectbox("Choose an action", ["Generate Contract", "Validate Contract"])

if option == "Generate Contract":
    st.header("Generate a Business Contract")
    num_contracts = st.number_input("Number of contracts to generate", min_value=1, max_value=50, value=1)

    if st.button("Generate"):
        for i in range(num_contracts):
            generated_contract = generate_template_contract()
            file_name = f"generated_contract_{i+1}.pdf"
            save_contract_as_pdf(generated_contract, file_name)
            st.success(f"Contract {i+1} generated: {file_name}")
            with open(file_name, "rb") as file:
                btn = st.download_button(
                    label=f"Download Contract {i+1}",
                    data=file,
                    file_name=file_name,
                    mime="application/pdf"
                )

if option == "Validate Contract":
    st.header("Validate a Business Contract")
    uploaded_file = st.file_uploader("Upload a PDF file", type="pdf")

    if uploaded_file is not None:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(uploaded_file.read())
            tmp_file_path = tmp_file.name

        extracted_text = extract_text_from_pdf(tmp_file_path)

        summary = summarize_contract(extracted_text)
        st.subheader("Contract Summary")
        for key, value in summary.items():
            st.write(f"**{key}**: {value}")

        deviation_percentage = check_deviation(extracted_text, generate_template_contract())
        st.subheader("Deviation from Template")
        st.write(f"Deviation: {deviation_percentage:.2f}%")

        important_details = [summary['parties'], summary['duration'], summary['payment_terms']]
        output_pdf_path = "highlighted_" + uploaded_file.name
        highlight_important_details(tmp_file_path, important_details, output_pdf_path)

        st.subheader("Highlighted Contract")
        with open(output_pdf_path, "rb") as file:
            st.download_button(
                label="Download Highlighted Contract",
                data=file,
                file_name=output_pdf_path,
                mime="application/pdf"
            )



Writing vandana.py


In [ ]:
!streamlit run vandana.py & npx localtunnel --port 8501

/bin/bash: line 1: streamlit: command not found
npx: installed 22 in 3.897s
your url is: https://eleven-plants-hunt.loca.lt
